In [1]:
import sys
sys.path.extend(['..', '../', '../..', '../../' ])
%load_ext autoreload
%autoreload 2

# LeNet5 - CALTECH101

In [2]:
import paths
import keras
from src.parameters import metrics
from src.datasets.dataset_generators import DatasetGenerators
from src.keras_callback.time_history import TimeHistory
from src.export.csv_export import export_learning_history_to_csv, export_learning_validation_result_to_csv
from src.export.model_export import clear_exported_models
from src.keras_model.lenet5_network import LeNet5Network

Using TensorFlow backend.


In [3]:
network_name = 'lenet5'
fit_epochs = 50
dataset = 'caltech101'
dataset_percentage = '40'

In [4]:
network = LeNet5Network()

In [5]:
dataset_generators = DatasetGenerators(
    name=f'{dataset}_{dataset_percentage}',
    dataset_dir=paths.data_location,
    image_size=network.image_size,
    batch_size=network.batch_size,
    grayscale=network.grayscale
)

Loading train set for caltech101_40...
Found 2451 images belonging to 101 classes.
Loading test set for caltech101_40...
Found 735 images belonging to 101 classes.
Loading validation set for caltech101_40...
Found 398 images belonging to 101 classes.


In [6]:
model = network.get_compiled_model(
    classes=dataset_generators.test.num_classes
)

In [7]:
checkpoint_path = f'{paths.models_location}{network_name}/{dataset}/'
best_model_path = f'{checkpoint_path}best.h5'
clear_exported_models(checkpoint_path)

checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=f'{checkpoint_path}{{epoch:02d}}-{{val_accuracy}}.h5',
    save_best_only=False,
    save_weights_only=False
)

best_callback = keras.callbacks.ModelCheckpoint(
    filepath=best_model_path,
    save_best_only=True,
    verbose=1,
    save_weights_only=False
)

time_callback = TimeHistory()

model_callbacks = [checkpoint_callback, best_callback, time_callback]

In [8]:
fit_result = model.fit_generator(
    generator=dataset_generators.train,
    epochs=fit_epochs,
    validation_data=dataset_generators.test,
    callbacks = model_callbacks
)

Epoch 1/50
77/77 [==============================] - 3s 42ms/step - loss: 5.8664 - accuracy: 0.1302 - precision: 0.1818 - recall: 0.1082 - val_loss: 4.4441 - val_accuracy: 0.2190 - val_precision: 0.2609 - val_recall: 0.2130
Epoch 2/50
77/77 [==============================] - 3s 36ms/step - loss: 3.7571 - accuracy: 0.2387 - precision: 0.4870 - recall: 0.3600 - val_loss: 2.9506 - val_accuracy: 0.2503 - val_precision: 0.5942 - val_recall: 0.5333
Epoch 3/50
77/77 [==============================] - 3s 35ms/step - loss: 3.4276 - accuracy: 0.2811 - precision: 0.6126 - recall: 0.5152 - val_loss: 4.3134 - val_accuracy: 0.2694 - val_precision: 0.6087 - val_recall: 0.5362
Epoch 4/50
77/77 [==============================] - 3s 35ms/step - loss: 3.1211 - accuracy: 0.3329 - precision: 0.7273 - recall: 0.6764 - val_loss: 3.8140 - val_accuracy: 0.3048 - val_precision: 0.6957 - val_recall: 0.6522
Epoch 5/50
77/77 [==============================] - 3s 35ms/step - loss: 2.8000 - accuracy: 0.3839 - precisi

Epoch 38/50
77/77 [==============================] - 3s 35ms/step - loss: 0.0924 - accuracy: 0.9755 - precision: 0.8312 - recall: 0.8268 - val_loss: 8.7133 - val_accuracy: 0.3524 - val_precision: 0.7464 - val_recall: 0.8696
Epoch 39/50
77/77 [==============================] - 3s 37ms/step - loss: 0.0584 - accuracy: 0.9865 - precision: 0.8268 - recall: 0.8247 - val_loss: 10.1556 - val_accuracy: 0.3537 - val_precision: 0.6254 - val_recall: 0.7029
Epoch 40/50
77/77 [==============================] - 3s 36ms/step - loss: 0.0542 - accuracy: 0.9869 - precision: 0.8312 - recall: 0.8268 - val_loss: 6.8904 - val_accuracy: 0.3361 - val_precision: 0.6268 - val_recall: 0.7391
Epoch 41/50
77/77 [==============================] - 3s 36ms/step - loss: 0.0479 - accuracy: 0.9869 - precision: 0.7760 - recall: 0.7792 - val_loss: 11.7455 - val_accuracy: 0.3497 - val_precision: 0.7246 - val_recall: 0.8116
Epoch 42/50
77/77 [==============================] - 3s 36ms/step - loss: 0.0555 - accuracy: 0.9869 - 

In [9]:
best_model = keras.models.load_model(best_model_path,
    custom_objects= {
        'precision': metrics.precision,
        'recall': metrics.recall
    }
)
validation_result = best_model.evaluate_generator(
    generator=dataset_generators.validate
)

## Results

### Epoch CSV

In [10]:
csv_file_path = f'{paths.results_location}{network_name}/{dataset}-history.csv'
export_learning_history_to_csv(csv_file_path, fit_result, time_callback.times)

epoch, time, val_loss, val_accuracy, val_precision, val_recall
0,0,0,0,0,0
1,3.2246956825256348,4.444108009338379,0.21904762089252472,0.260869562625885,0.2130434662103653
2,2.787241220474243,2.9505507946014404,0.25034013390541077,0.5942028760910034,0.5333333015441895
3,2.7122058868408203,4.313374996185303,0.26938775181770325,0.6086956262588501,0.5362318754196167
4,2.704558849334717,3.813999891281128,0.3047619163990021,0.695652186870575,0.6521739363670349
5,2.707820177078247,3.660388469696045,0.28707483410835266,0.8478260636329651,0.8586956262588501
6,2.703456401824951,3.796356678009033,0.3142857253551483,0.7137681841850281,0.782608687877655
7,2.8965978622436523,3.9487476348876953,0.3292517066001892,0.7898550629615784,0.804347813129425
8,2.7329893112182617,3.322382926940918,0.3306122422218323,0.6072463393211365,0.739130437374115
9,2.8742599487304688,5.229778289794922,0.3401360511779785,0.6775361895561218,0.782608687877655
10,2.7924270629882812,3.239232301712036,0.3469387888908386,0.7717

### Validation result

In [11]:
csv_file_path = f'{paths.results_location}{network_name}/{dataset}-result.csv'
export_learning_validation_result_to_csv(csv_file_path, validation_result, time_callback.times)

avg_epoch_time, time, val_loss, val_accuracy, val_precision, val_recall
2.8211167860031128,141.05583930015564,9.706934928894043,0.36432161927223206,0.5384615659713745,0.6153846383094788
